<a href="https://colab.research.google.com/github/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/Train_YOLO_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 중고차 육안검사 보조 AI서비스

**작성자:** 이지홍

**GitHub:** [car-defect-check](https://github.com/lee-ji-hong/car-defect-check)

### 소개

이 노트북은 [Ultralytics YOLO](https://docs.ultralytics.com/) 프레임워크를 활용하여  
**차량 외관의 스크래치, 찌그러짐, 균열 등의 하자를 자동으로 탐지하는 객체 탐지 모델**을 학습합니다. 

노트북을 끝까지 따라 하면, 실제 차량 이미지에서 하자를 인식할 수 있는 맞춤형 YOLOv8 모델을 완성할 수 있습니다.  
이 모델은 PC, 엣지 디바이스, 또는 모바일 기기 등에서 실시간으로 활용할 수 있습니다.

<p align=center>
<img src="" height="360"><br>
<i>차량 외관 손상 부위를 인식하는 모델의 출력 결과</i>
</p>

이 노트북은 Label Studio 등에서 YOLO 형식으로 라벨링된 이미지 데이터를 기반으로 학습합니다.  

---

### 🚘 목적: 차량 자동 외관 검사

이 프로젝트는 다음과 같은 목적에 활용할 수 있습니다:

- 자동차 정비소, 렌터카 반납 시 **자동 하자 검사**
- 중고차 판매 플랫폼에서 **신뢰성 있는 외관 평가**
- 키오스크 및 모바일 앱을 활용한 **비대면 진단 시스템 구축**


### ✅ 이 노트북에서 다루는 내용

- **프레임워크**: Ultralytics YOLOv8  
- **학습 목적**: 객체 탐지를 통한 차량 하자 인식  
- **데이터셋**: 사용자 정의 이미지 + YOLO 라벨 형식  
- **산출물**: 실사용 가능한 `best.pt` 모델 파일


### 💻 Colab 환경에서 실행하기

Google Colab은 웹 브라우저에서 실행 가능한 Linux 가상 환경을 제공하며,  
Python 및 GPU를 무료로 사용할 수 있습니다.  

이 노트북에서는 다음 작업을 수행합니다:

- PyTorch 및 Ultralytics 설치
- 데이터셋 업로드 및 구성
- YOLOv8 모델 학습
- 결과 확인 및 모델 다운로드

▶️ 각 코드 셀 오른쪽의 **재생 버튼**을 눌러 순서대로 실행하면 됩니다.


### 🧭 노트북 구성 안내

좌측 사이드바의 목차를 활용하여 아래 주요 섹션으로 이동할 수 있습니다:

- 0. 기본 설정  
- 1. 라이브러리 설치  
- 2. 데이터셋 업로드 및 준비  
- 3. YOLOv8 모델 학습  
- 4. 결과 평가 및 시각화  
- 5. 추론 예제 실행  
- 6. 학습된 모델 다운로드

---


**NVIDIA GPU 가용성 확인**

상단 메뉴 막대에서 "런타임" -> "런타임 유형 변경"으로 이동한 다음 "하드웨어 가속기" 섹션에서 GPU 옵션 중 하나를 선택하여 GPU가 장착된 컴퓨터를 사용하고 있는지 확인하세요. 다음 코드 블록에서 "재생"을 클릭하여 NVIDIA GPU가 존재하고 학습할 준비가 되었는지 확인하세요.

In [ ]:
!nvidia-smi

#1.&nbsp;훈련 이미지 수집 및 레이블 지정

학습을 시작하기 전에 객체 감지 모델 학습에 사용할 이미지를 수집하고 레이블을 지정해야 합니다. 개념 증명 모델을 위한 좋은 시작점은 200개의 이미지입니다. 학습 이미지는 원하는 객체와 함께 무작위 객체가 포함되어 있어야 하며, 다양한 배경과 조명 조건을 가져야 합니다.

이미지를 수집하는 데에는 몇 가지 옵션이 있습니다.

- 객체의 사진을 직접 찍고 레이블을 지정하여 사용자 지정 데이터 세트를 구축합니다(일반적으로 이렇게 하면 최상의 성능을 얻을 수 있습니다).

자신만의 데이터 세트를 만들고 싶다면 이미지에 레이블을 지정하는 여러 도구를 사용할 수 있습니다. 그중 한 가지 좋은 선택지는 무료 오픈 소스 레이블 지정 도구인 Label Studio 입니다 . 이 도구는 간단한 워크플로를 제공하면서도 고급 기능을 위한 기능을 제공합니다. 이 노트북을 안내하는 제 YouTube 동영상(링크는 곧 추가될 예정입니다)에서는 Label Studio를 사용하여 이미지에 레이블을 지정하는 방법을 보여줍니다.

<p align=center>
<img src="https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/doc/label-studio-example.PNG" height="380"><br>
<i>Label Studio로 라벨이 지정된 사탕 이미지의 예입니다.</i>
</p>

project.zipLabel Studio를 사용하여 이미지에 레이블을 지정하고 내보낸 경우 다음 내용이 포함된 파일 로 내보내집니다 .

- images이미지가 들어있는 폴더
- labelsYOLO 주석 형식의 레이블을 포함하는 폴더
- classes.txt모든 클래스를 포함하는 레이블맵 파일
- Label Studio에 대한 특정 정보가 포함된 파일 notes.json(이 파일은 무시할 수 있음)

다른 소스(예: Roboflow Universe)에서 데이터 세트를 얻었거나 다른 도구를 사용하여 데이터 세트에 레이블을 지정한 경우 파일이 동일한 폴더 구조로 구성되어 있는지 확인합니다.

<p align=center>
<img src="https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/doc/zipped-data-example.png" height=""><br>
</p>

데이터 세트를 구축하고 위에 표시된 파일 구조에 넣은 후 압축하면 data.zip다음 단계로 넘어갈 준비가 된 것입니다.

# 2.&nbsp;이미지 데이터셋 업로드 및 학습 데이터 준비

다음으로, 데이터 세트를 업로드하고 YOLO 학습을 위해 준비합니다. 데이터 세트를 학습 폴더와 검증 폴더로 나누고, 모델 학습을 위한 구성 파일을 자동으로 생성합니다.

## 2.1 이미지 업로드

먼저, Colab에 데이터 세트를 업로드해야 합니다. data.zip폴더를 이 Colab 인스턴스로 옮기는 몇 가지 방법은 다음과 같습니다.

**Option 1. Google Colab을 통해 업로드**

`data.zip` 브라우저 왼쪽의 "파일" 아이콘을 클릭한 다음 "세션 저장소에 업로드" 아이콘을 클릭하여 Google Colab 인스턴스에 파일을 업로드하세요 . zip 파일을 선택하여 업로드하세요.

<p>
<br>
<img src="https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/doc/upload-colab-files.png" height="240">
</p>



**Option 2. Google 드라이브에서 복사**

이미지를 개인 Google 드라이브에 업로드하고, 이 Colab 세션에 드라이브를 마운트한 후 Colab 파일 시스템에 복사할 수도 있습니다. 이 옵션은 Colab을 재시작할 때마다 업로드될 때까지 기다릴 필요 없이 이미지를 미리 업로드하려는 경우에 유용합니다. 이미지 용량이 50MB가 넘는 경우 이 옵션을 사용하는 것이 좋습니다.

먼저, `data.zip`파일을 Google Drive에 업로드하고 업로드한 폴더를 기록해 두세요. `MyDrive/path/to/data.zip` 파일 경로로 바꾸세요. (예를 들어, "candy-dataset1" 폴더에 zip 파일을 업로드했으므로 `MyDrive/candy-dataset1/data.zip`경로에 를 사용합니다.) 그런 다음, 다음 코드 블록을 실행하여 Google Drive를 이 Colab 세션에 마운트하고 폴더를 이 파일 시스템에 복사합니다.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/MyDrive/path/to/data.zip /content

## 2.2 이미지를 학습 폴더와 검증 폴더로 분할

이 시점에서 옵션 1, 2, 3 중 어떤 옵션을 사용하셨든 왼쪽의 폴더 아이콘을 클릭하면 `data.zip`파일 목록에 파일이 표시됩니다. 다음으로, 압축을 풀고 `data.zip`이미지를 저장할 폴더를 생성하겠습니다. 다음 코드 블록을 실행하여 데이터의 압축을 해제하세요.

In [ ]:
# Unzip images to a custom data folder
!unzip -q /content/data.zip -d /content/custom_data

Ultralytics는 모델의 학습 데이터를 저장하기 위해 특정 폴더 구조를 필요로 합니다. Ultralytics는 모델의 학습 데이터를 저장하기 위해 특정 폴더 구조를 필요로 합니다. 루트 폴더의 이름은 "data"입니다. 내부에는 두 개의 주요 폴더가 있습니다.

*   **학습**: 모델을 학습하는 데 사용되는 실제 이미지입니다. 한 번의 학습 에포크(epoch) 동안 학습 세트의 모든 이미지가 신경망에 전달됩니다. 학습 알고리즘은 이미지의 데이터에 맞게 네트워크 가중치를 조정합니다.


*   **검증**: 이러한 이미지는 각 학습 에포크가 끝날 때마다 모델의 성능을 확인하는 데 사용됩니다.

각 폴더에는 "images" 폴더와 "labels" 폴더가 있으며, 각각 이미지 파일과 주석 파일을 보관합니다.


필요한 폴더 구조를 자동으로 생성하고 데이터세트의 90%를 "train" 폴더로, 10%를 "validation" 폴더로 무작위로 이동하는 Python 스크립트를 작성했습니다. 다음 코드 블록을 실행하여 스크립트를 다운로드하고 실행하세요.

In [ ]:
!wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

# TO DO: Improve robustness of train_val_split.py script so it can handle nested data folders, etc
!python train_val_split.py --datapath="/content/custom_data" --train_pct=0.9

# 3.&nbsp;설치 요구 사항(Ultralytics)

다음으로, 이 Google Colab 인스턴스에 Ultralytics 라이브러리를 설치하겠습니다. 이 Python 라이브러리는 YOLO 모델을 학습하는 데 사용됩니다.

In [ ]:
!pip install ultralytics

# 4.&nbsp;Configure Training


학습을 실행하기 전에 마지막 단계가 있습니다. Ultralytics 학습 구성 YAML 파일을 생성해야 합니다. 이 파일은 학습 및 검증 데이터의 위치를 ​​지정하고 모델의 클래스도 정의합니다. 

Run the code block below to automatically generate a `data.yaml` configuration file. Make sure you have a labelmap file located at `custom_data/classes.txt`. If you used Label Studio or one of my pre-made datasets, it should already be present. If you assembled the dataset another way, you may have to manually create the `classes.txt` file (see [here](https://github.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/doc/classes.txt) for an example of how it's formatted).

아래 코드 블록을 실행하여 `data.yaml`구성 파일을 자동으로 생성하세요. labelmap 파일이 있는지 확인하세요 `custom_data/classes.txt`. Label Studio을 사용했다면 이미 존재할 것입니다. 다른 방식으로 데이터세트를 구성했다면 파일을 수동으로 생성해야 할 수도 있습니다.

In [ ]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

# 5.&nbsp;모델 학습

## 5.1 Training Parameters
이제 데이터가 정리되고 설정 파일이 생성되었으니 학습을 시작할 준비가 되었습니다! 먼저, 몇 가지 중요한 매개변수를 결정해야 합니다.

**모델 아키텍처 및 크기 (`model`):**

yolo11n.pt학습에 사용할 수 있는 YOLO11 모델 크기는 `yolo11n.pt`, `yolo11s.pt`, `yolo11m.pt`, `yolo11l.pt`,`yolo11xl.pt` 를 포함하여 여러 가지가 있습니다. 모델 크기가 클수록 실행 속도는 느리지만 정확도는 높고, 모델 크기가 작을수록 실행 속도는 빠르지만 정확도는 낮습니다. 어떤 모델 크기를 사용해야 할지 확실하지 않다면 `yolo11s.pt`를 사용하는 것이 좋습니다.

또는 `yolo11`을  대입하여 `yolov8` or `yolov5` 모델을 훈련할 수도 있습니다 .


**에포크 수 (`epochs`)**

머신러닝에서 한 "에포크"는 전체 학습 데이터셋을 한 번 통과하는 횟수입니다. 에포크 횟수는 모델이 학습할 시간을 결정합니다. 최적의 에포크 횟수는 데이터셋의 크기와 모델 아키텍처에 따라 달라집니다. 데이터셋에 이미지가 200개 미만인 경우 60에포크로 시작하는 것이 좋습니다. 데이터셋에 이미지가 200개 이상인 경우 40에포크로 시작하는 것이 좋습니다.


**해상도 (`imgsz`)**

해상도는 모델의 속도와 정확도에 큰 영향을 미칩니다. 해상도가 낮은 모델은 속도는 빠르지만 정확도는 떨어집니다. YOLO 모델은 일반적으로 640x640 해상도에서 학습 및 추론됩니다. 하지만 모델 실행 속도를 높이거나 저해상도 이미지를 사용할 예정이라면 480x480과 같은 낮은 해상도를 사용해 보세요.


## 5.2 Run Training

 다른 모델, 에포크 수 또는 해상도를 사용하려면 , `model`, `epochs`, or `imgsz`를 변경하세요.

In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=60 imgsz=640

학습 알고리즘은 학습 및 검증 디렉터리의 이미지를 파싱한 후 모델 학습을 시작합니다. 각 학습 에포크가 끝날 때마다 프로그램은 검증 데이터셋을 사용하여 모델을 실행하고 결과 mAP, 정밀도, 재현율을 보고합니다. 학습이 진행됨에 따라 mAP는 일반적으로 매 에포크마다 증가합니다. 에포크 수에 지정된 횟수를 지나면 학습이 종료됩니다.

> **참고:** 최적화 프로그램은 모델에서 불필요한 레이어를 제거하기 위해 학습이 끝날 때 실행되므로 학습이 완료될 때까지 실행해야 합니다.

가장 잘 훈련된 모델 가중치는 `content/runs/detect/train/weights/best.pt`폴더에 저장됩니다. 각 에포크에서 손실, 정밀도, 재현율, mAP가 어떻게 진행되었는지 보여주는 `results.png`파일을 포함하여 훈련에 대한 추가 정보가 `content/runs/detect/train`폴더에 저장됩니다 .

#6.&nbsp;테스트 모델

이제 모델이 학습되었습니다. 아래 명령어는 검증 폴더의 이미지에 대해 모델을 실행한 후 처음 10개 이미지의 결과를 표시합니다. 이는 모델이 예상대로 작동하는지 확인하는 좋은 방법입니다. 아래 블록의 "재생"을 클릭하여 모델 성능을 확인할 수 있습니다.

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')


모델은 각 이미지에서 관심 있는 각 객체 주위에 상자를 그려야 합니다. 객체 감지가 제대로 이루어지지 않는 경우, 다음과 같은 몇 가지 팁을 참고하세요.

1. 데이터 세트를 두 번 확인하여 레이블 오류나 상충되는 예가 없는지 확인하세요.
2. 훈련에 사용되는 에포크 수를 늘립니다.
3. 더 큰 모델 크기를 사용하세요(예: `yolo11l.pt`).

비디오 파일이나 다른 이미지 파일을 이 노트북에 업로드하고 위 `!yolo detect predict` 명령을 사용하여 모델을 실행할 수도 있습니다 . 여기서 `source`는 비디오 파일, 이미지 또는 이미지 폴더의 위치를 ​​가리킵니다. 결과는 `runs/detect/predict`에 저장됩니다.

이미지에 상자를 그리는 것은 훌륭하지만, 그 자체로는 그다지 유용하지 않습니다. 또한 이 모델을 Colab 노트북에서 실행하는 것도 그다지 유용하지 않습니다. 로컬 컴퓨터에서 실행하는 것이 더 쉽습니다. 새로 학습된 모델을 다운로드하고 로컬 기기에서 실행하는 방법은 다음 섹션으로 넘어가세요.

#7.&nbsp;모델 배포

이제 사용자 지정 모델이 학습되었으므로 애플리케이션에 다운로드하여 배포할 준비가 되었습니다! YOLO 모델은 PC, 임베디드 시스템, 휴대폰 등 다양한 하드웨어에서 실행될 수 있습니다. Ultralytics를 사용하면 YOLO 모델을 다양한 형식(`tflite`, `onnx`, etc.)으로 쉽게 변환하고 다양한 환경에 배포할 수 있습니다.

## 7.1 YOLO 모델 다운로드

먼저, 아래 코드 블록을 실행하여 학습된 모델을 압축하여 다운로드하세요.

이 코드는 `my_model`이라는 이름의 폴더를 만들고, 모델 가중치를 그 안에 넣은 후, 이름을 `best.pt`에서 `my_model.pt`로 바꿉니다. 또한 나중에 참조할 수 있도록 학습 결과도 추가합니다. 그런 다음 폴더를 `my_model.zip`으로 압축합니다.

In [ ]:
# Create "my_model" folder to store model weights and train results
!mkdir /content/my_model
!cp /content/runs/detect/train/weights/best.pt /content/my_model/my_model.pt
!cp -r /content/runs/detect/train /content/my_model

# Zip into "my_model.zip"
%cd my_model
!zip /content/my_model.zip my_model.pt
!zip -r /content/my_model.zip train
%cd /content

In [ ]:
# This takes forever for some reason, you can also just download the model from the sidebar
from google.colab import files

files.download('/content/my_model.zip')

## 7.2 로컬 장치에 YOLO 모델 배포

다음으로, 다운로드한 모델을 로컬 기기에서 실행해 보겠습니다. 이 섹션에서는 다양한 기기에 YOLO 모델을 배포하는 방법을 설명합니다.

`yolo_detect.py`모델을 로드하고, 이미지 소스에 추론을 실행하고, 추론 결과를 파싱하고, 이미지에서 감지된 각 클래스 주위에 상자를 표시하는 방법을 보여주는 기본적인 Python 스크립트를 작성했습니다.

### 7.2.1 PC(Windows, Linux 또는 macOS)에 배포

PC에서 Ultralytics 모델을 실행하는 가장 쉬운 방법은 Anaconda를 사용하는 것입니다. Anaconda는 가상 Python 환경을 설정하고 Ultralytics와 PyTorch를 쉽게 설치할 수 있도록 지원합니다. CUDA와 cuDNN도 자동으로 설치되므로 NVIDIA GPU를 사용하여 모델 추론 속도를 높일 수 있습니다.

**1. Anaconda 다운로드 및 설치**

anaconda 혹은 minianaconda 설치

**2. 가상 환경 설정**

설치가 완료되면 시작 메뉴에서 Anaconda Prompt를 실행하세요. (macOS나 Linux를 사용하는 경우, 명령 터미널을 열면 됩니다.)

다음 명령을 실행하여 새로운 Python 환경을 만들고 활성화합니다:

```
conda create --name yolo-env1 python=3.12 -y
conda activate yolo-env1
```

다음 명령을 실행하여 Ultralytics를 설치합니다(OpenCV-Python, Numpy, PyTorch와 같은 가져오기 라이브러리도 설치합니다):

```
pip install ultralytics
```

NVIDIA GPU가 있는 경우 다음 명령을 실행하여 PyTorch의 GPU 지원 버전을 설치할 수 있습니다.(없으면 그냥 cpu로 진행):

```
pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
```

**3. 다운로드한 모델 압축 해제**
7.1단계에서 다운로드한 `my_model.zip`파일을 PC의 폴더에 압축 해제합니다. Anaconda Prompt 터미널에서 다음을 사용하여 압축 해제된 폴더로 이동합니다:

```
cd path/to/folder
```

**4. yolo_detect.py를 다운로드하고 실행하세요**

다음을 사용하여 `yolo_detect.py`스크립트를 `my_model`폴더에 다운로드하세요:

```
curl -o yolo_detect.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/yolo_detect.py
```

1280x720 해상도의 USB 카메라에서 Yolov8s 모델을 사용하여 추론을 실행하려면 다음 명령어를 실행하세요:

```
python yolo_detect.py --model my_model.pt --source usb0 --resolution 1280x720
```

웹캠에서 실시간으로 전송되는 영상을 보여주는 창이 나타나고, 각 프레임에서 감지된 객체 주위에 상자가 그려집니다.

비디오 파일, 이미지 또는 이미지 폴더에서도 모델을 실행할 수 있습니다.

### 7.2.2 Raspberry Pi에 배포

YOLO 모델을 NCNN 형식으로 변환하고 Raspberry Pi에서 실행